In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import batman

# Bayes' Theorem:

# $P(\theta|D) = \frac{P(D|\theta)P(\theta)}{P(D)}$

---

# P($\theta$|D) is the posterior
- the probability of a set of model parameters ($\theta$) given our data (D).

# P(D|$\theta$) is the likelihood
- how well a given model with parameters, $\theta$, fits the data.

# P($\theta$) is the prior
- our prior belief in the model parameters. This is usually based on what boundaries you think are physically reasonable for the system. Priors can be uniform (equal probability for all parameter values), gaussian, poisson, etc 

# P(D) is a normalization factor
- usually this is very difficult or impossible to calculate, so we marginalize over it

---


When we are using Bayesian statistics in our MCMC, it is helpful to take the log of this expression:

# $log(posterior) = log(likelihood) + log(prior) - log(normalization)$
so that we can deal with simpler math

---

# Markov-Chain Monte Carlo

This notebook isn't explicitly aiming to cover all of the fundamental statistics beneath emcee or model fitting in general, so I recommend you read [these](https://arxiv.org/pdf/1202.3665.pdf) [papers](https://arxiv.org/abs/1008.4686).

MCMC is a process by which you can sample the "true" posterior distribution of parameter space as it is explored by "walkers". Basically your multi-dimensional probability space (let's say radius, mid-transit time, and flux baseline for an exoplanet transit) is populated with some number of "walkers" that explore the space and calculate a prior and a likelihood for each set of model parameters. The walker examines the probability of those model parameters and then makes a stochastic jump to a new set of parameters and examines how the new parameters compare. By doing this stochastically, that is, with an element of randomness determining where the walker moves to next, you can create a sample of the posterior distribution of your parameters and therefore fit a model to your data.

The general idea is that we want to maximize the posterior probability of our parameters, which we can do by maximizing the likelihood and the prior. We aren't searching for a single "best fit" set of parameters, we're creating a probability distributions of our model parameters to better understand our model uncertainties.

---

# [emcee](https://emcee.readthedocs.io/en/stable/): an affine invariant* ensemble sampler for MCMC

emcee is a package which helps us to run these complicated multi-dimensional explorations of probabiltiy space by only having to tune a few parameters. This package uses a Metropolis-Hastings algorithm for sampling small PDFs at any given point in parameter space.

*Affine Invariant means this algorithm performs equally well under all linear transformations and is therefore insensitive to covariances among parameters.

---

# Let's simulate some transit data to analyze!

Say we are observing a transit of TOI 237b:

rp = 1.4 Rearth

rs = 0.21 Rsun

period = 5.4 days

In [ ]:
def BATMAN_MODEL(rp,
                 t0,
                 baseline = 1.0,
                 t = None):
    
    Rp = (rp/0.21)*(1/109) # Converts Rp in earth radii to dimensionless rp/r*
                    
    params = batman.TransitParams()
    params.t0 = t0                       # time of inferior conjunction ()
    params.rp = Rp                       # planet radius (in units of stellar radii)
    params.per = 5.4                     # period in days -> time array MUST BE THE SAME UNITS
    params.a = 34.7                      # semi-major axis (in units of stellar radii)
    params.inc = 89.5                    # orbital inclination (in degrees)
    params.ecc = 0.                      # eccentricity
    params.w = 90.                       # longitude of periastron (in degrees)
    params.u = [0.27,0.29]               # limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"       # limb darkening model
        
    m = batman.TransitModel(params, t)   # initializes model
    
    flux = m.light_curve(params)*baseline   # calculates light curve
    
    return flux

In [ ]:
# set up a time array which is 6 hours long
start_hr = -2.0
stop_hr = 2.0
length = 200
times = np.linspace(start_hr/24,stop_hr/24,length) #converting the time array in hours to days

# Now using the batman package, we can simulate an idealized transit
sim_data = BATMAN_MODEL(rp = 1.4, t0=0, baseline = 1.0,  t = times)
noise = 0.002 # 2ppt noise, about what I can get with LCO 1m observations
scatter = np.random.normal(0,noise,length)

data = sim_data + scatter
err = np.linspace(0.002,0.002,length)

plt.figure(figsize=(10,6))
plt.errorbar(times,data,yerr=err,zorder=10000,alpha=0.5,label='simulated data',fmt='o')
plt.legend(loc='lower left')
plt.xlabel('Time from mid-transit (hr)')
plt.ylabel('Relative Flux')
plt.show()

# Now before we start using Emcee, let's define a log posterior function, using a Gaussian likelihood and uniform priors

In [ ]:
def lnprob(parameters, plot=False):
        
    # pull out some model parameters
    Rp, t0, Baseline = parameters
    
    model_flux = BATMAN_MODEL(Rp, t0, Baseline, t = times)
    
    if plot:
        plt.errorbar(times,data,yerr=err,fmt='o',alpha = 0.5,label='Simulated Data')
        plt.plot(times,model_flux,label='MCMC Model',color='k',zorder=100)
        plt.xlabel('Hours from Mid-Transit')
        plt.ylabel('Flux')
        
        plt.legend(frameon=False)    
        plt.show()
        
    
    # let's put some hard limits on parameter space in place of more complicated priors
    
    if (0.0 < Baseline) and (0.0 <= Rp) and (-2.0 < t0 < 2.0 ):
        chisq = np.sum((data - model_flux)**2/(err)**2)
        lnp = np.sum(1/np.sqrt(2*np.pi*(err))) - 0.5*chisq #this is a gaussian likelihood
    
        return lnp
    
    return -np.inf

Test that the lnprob, with a given set of (true) parameters, returns the expected model.

In [ ]:
params = [1.4, 0.0, 1.0] # [rp, t0, baseline]

lnprob(params,plot=True)

In [ ]:
# intialize some walkers
ndim, nwalkers, nsteps = 3, 100, 3000

# these are initial parameters
baseline_init = np.random.uniform(0.9, 1.1, nwalkers)
rp_init = np.random.uniform(0, 3, nwalkers)
t0_init = np.random.uniform(-0.05, 0.05, nwalkers)

p0 = np.transpose([rp_init, t0_init, baseline_init])

In [ ]:
# create a sampler and run it
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)
result = sampler.run_mcmc(p0, nsteps)

In [ ]:
burnin = int(0.1*nsteps) # let's toss out the first 10% of samples as the "burn-in"
Rp, t0, Base = sampler.chain.T
Rp_trimmed, t0_trimmed, Base_trimmed = sampler.chain.T[:, burnin:nsteps, :]

rp_samples = Rp_trimmed.flatten()
t0_samples = t0_trimmed.flatten()
baseline_samples = Base_trimmed.flatten()

# Let's plot our data and some random samples from the sampler to see how well the MCMC worked!

In [ ]:
plt.figure(figsize=(12,8))
plt.errorbar(times,data,yerr=err,fmt='o',alpha = 0.5,label='Simulated Data')
plt.xlabel('Hours from Mid-Transit')
plt.ylabel('Flux')

#########################################
# Plot 200 random models from the samples
#########################################
    
for j in range(0,200,1):
    i = np.random.randint(low=0,high=(nsteps-burnin)*nwalkers)
    sampled_model = BATMAN_MODEL(rp = rp_samples[i], t0 = t0_samples[i], baseline = baseline_samples[i],  t = times)
    plt.plot(times,sampled_model,color='darkred',alpha = 0.1,
            linewidth=0.8,zorder=100)

plt.show()

In [ ]:
def plot_chain(start=0, stop=nsteps):
    '''Plot the chain, in a couple different ways.'''

    plt.figure(figsize=(14,14))
    gs = plt.matplotlib.gridspec.GridSpec(4,2, hspace=0.6)
    
    #Walker Plots
    ax_Base = plt.subplot(gs[0,0])
    ax_Base.plot(Base.flatten()[::nwalkers],color='black',alpha=0.5); 
    ax_Base.axvspan(start, stop, zorder=-1,alpha=0.3); 
    ax_Base.set_ylabel('Baseline Flux')
    
    ax_Rp = plt.subplot(gs[0,1], sharex=ax_Base)
    ax_Rp.plot(Rp.flatten()[::nwalkers],color='black',alpha=0.5)
    ax_Rp.axvspan(start, stop, zorder=-1,alpha=0.3)
    ax_Rp.set_ylabel('Rp')
    
    ax_t0 = plt.subplot(gs[1,0], sharex=ax_Base)
    ax_t0.plot(t0.flatten()[::nwalkers],color='black',alpha=0.5)
    ax_t0.axvspan(start, stop, zorder=-1,alpha=0.3)
    ax_t0.set_ylabel('Mid-Transit Time (hr)')
    
    #Histograms
    ax_Basehist = plt.subplot(gs[1,1])
    ax_Basehist.hist(Base.flatten(),color='black',bins=60)
    ax_Basehist.axvline(1.0,zorder=100,color='purple')
    ax_Basehist.set_xlabel('Baseline')
    
    ax_Rphist = plt.subplot(gs[2,0])
    ax_Rphist.hist(Rp.flatten(),color='black',bins=60)
    ax_Rphist.set_xlabel('Rp')
    
    ax_t0hist = plt.subplot(gs[2,1])
    ax_t0hist.hist(t0.flatten(),color='black',bins=60)
    ax_t0hist.set_xlabel('Mid-Transit')

In [ ]:
plot_chain(burnin,nsteps)